# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
import os
if os.path.exists('api_keys.py'):
    from api_keys import geoapify_key
else:
    geoapify_key = 'YOUR_API_KEY_HERE'  # Replace with your actual API key
try:
    import hvplot.pandas
except ImportError:
    !pip install hvplot
    import hvplot.pandas
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,yellowknife,62.4560,-114.3525,-3.71,80,100,6.69,CA,1730175890
1,1,zarumilla,-3.5031,-80.2731,22.27,82,100,1.70,PE,1730175891
2,2,badger,64.8000,-147.5333,-5.57,87,100,1.54,US,1730175892
3,3,kismayo,-0.3582,42.5454,26.16,74,62,3.97,SO,1730175893
4,4,waitangi,-43.9535,-176.5597,14.34,85,28,1.79,NZ,1730175894


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [17]:
# Configure the map plot
city_map = city_data_df.hvplot.points("Lng",
                                      "Lat",
                                      geo=True,
                                      size="Humidity",  # This will use the humidity value for the point size
                                      color="City",
                                      tiles="OSM",
                                      frame_width=700,
                                      frame_height=500,
                                      title="City Humidity Levels")

# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [19]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[
    (city_data_df["Max Temp"] >= 17) & 
    (city_data_df["Max Temp"] <= 22) & 
    (city_data_df["Humidity"] < 60) &
    (city_data_df["Wind Speed"] < 4.5) &
    (city_data_df["Cloudiness"] < 20)
]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
22,22,san luis,-33.2950,-66.3356,21.45,38,7,3.58,AR,1730175644
80,80,mount gambier,-37.8333,140.7667,18.47,53,12,4.14,AU,1730175984
81,81,dongcun,36.7767,121.1597,17.19,40,0,1.85,CN,1730175985
90,90,blackmans bay,-43.0167,147.3167,20.85,47,13,2.70,AU,1730175997
173,173,mingshui,36.7167,117.5000,20.35,39,0,2.84,CN,1730176170
181,181,gejiu,23.3060,103.1635,17.76,59,6,1.41,CN,1730176179
217,217,chios,38.3678,26.1358,19.92,32,0,3.60,GR,1730176223
222,222,port lincoln,-34.7333,135.8667,17.72,58,0,3.96,AU,1730176230
248,248,kaduna,10.5222,7.4383,20.00,38,9,2.29,NG,1730176099
262,262,arroyito,-31.4202,-63.0500,21.06,55,5,1.79,AR,1730176280


### Step 3: Create a new DataFrame called `hotel_df`.

In [20]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
22,san luis,AR,-33.2950,-66.3356,38,
80,mount gambier,AU,-37.8333,140.7667,53,
81,dongcun,CN,36.7767,121.1597,40,
90,blackmans bay,AU,-43.0167,147.3167,47,
173,mingshui,CN,36.7167,117.5000,39,
181,gejiu,CN,23.3060,103.1635,59,
217,chios,GR,38.3678,26.1358,32,
222,port lincoln,AU,-34.7333,135.8667,58,
248,kaduna,NG,10.5222,7.4383,38,
262,arroyito,AR,-31.4202,-63.0500,55,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [22]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories": "accommodation.hotel",  # Filter for hotels
          "limit": 1,                           # Limit results to 1 hotel
          "apiKey": geoapify_key}               # Your Geoapify API key

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
san luis - nearest hotel: Hotel Dos Venados
mount gambier - nearest hotel: The Old Mount Gambier Gaol
dongcun - nearest hotel: 烟台海阳方圆大酒店
blackmans bay - nearest hotel: Villa Howden
mingshui - nearest hotel: No hotel found
gejiu - nearest hotel: No hotel found
chios - nearest hotel: Agora Residence
port lincoln - nearest hotel: Boston Hotel
kaduna - nearest hotel: Crocodile Hotel
arroyito - nearest hotel: MH HOTEL
tanumah - nearest hotel: No hotel found
bardai - nearest hotel: No hotel found
al hillah - nearest hotel: فندق الاسراء السياحي


,City,Country,Lat,Lng,Humidity,Hotel Name
22,san luis,AR,-33.2950,-66.3356,38,Hotel Dos Venados
80,mount gambier,AU,-37.8333,140.7667,53,The Old Mount Gambier Gaol
81,dongcun,CN,36.7767,121.1597,40,烟台海阳方圆大酒店
90,blackmans bay,AU,-43.0167,147.3167,47,Villa Howden
173,mingshui,CN,36.7167,117.5000,39,No hotel found
181,gejiu,CN,23.3060,103.1635,59,No hotel found
217,chios,GR,38.3678,26.1358,32,Agora Residence
222,port lincoln,AU,-34.7333,135.8667,58,Boston Hotel
248,kaduna,NG,10.5222,7.4383,38,Crocodile Hotel
262,arroyito,AR,-31.4202,-63.0500,55,MH HOTEL


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [25]:
# Configure the map plot
map_plot = hotel_df.hvplot.points("Lng",
                                  "Lat",
                                  geo=True,
                                  tiles="OSM",
                                  size="Humidity",  # Size of the point based on humidity
                                  color="City",  # Different colors for different cities
                                  hover_cols=["City", "Country", "Hotel Name"],  # Add hover info
                                  title="Ideal City Locations with Hotel Information",
                                  xlabel="Longitude",
                                  ylabel="Latitude")

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)